In [2]:
def my_chat_reader(text_file_path):
    
    # On the 1st step, we will read the raw data of the chat, and split it by lines into a list
    my_text_opened = open(text_file_path, 'r', encoding='utf-8')
    my_text_to_read = my_text_opened.read()
    my_lines_list = my_text_to_read.split('\n')
    
    # On the 2nd step, we will make sure that the rows in the text are seperated correctly.
    # If the same message has more than 1 row, and therefore was splitted into several rows, it will be 1 row again. 
    # My side function will do that.
    my_fixed_lines_list = lines_fixed(my_lines_list)
    
    list_of_chat_dicts = []
    j=1
    dict_of_id = {}
    # On the 3rd step, we will seperate the data of each row into 3 groups, 1=date&time 2=id 3=text.
    for line in my_fixed_lines_list: 
        my_dict_to_sort = {}
        my_dict_to_sort["datetime"] = line[:15]
        # First check - if the text part is a message or data:
        rest_of_str = line[18:]
        text_index = rest_of_str.find(':')
        if text_index>=0:
            if '<המדיה לא נכללה>' in rest_of_str:  # Avoid this line
                continue
            num_start = rest_of_str.find('+')
            if num_start<0:
                non_anonymous_id = rest_of_str[:text_index] # Making the chat anonymous from names.
            else:
                non_anonymous_id = rest_of_str[num_start:num_start+16] # Making the chat anonymous from phone numbers.
            if non_anonymous_id not in dict_of_id:
                dict_of_id[non_anonymous_id] = (len(dict_of_id)+1)
            my_dict_to_sort["id"] = dict_of_id[non_anonymous_id]
            my_dict_to_sort["text"] = rest_of_str[text_index+1:]
            list_of_chat_dicts.append(my_dict_to_sort)
        else:  # For the data line.
            if j==1 or rest_of_str.find("הוסיף/ה")>=0: # Avoid unnecessary lines
                j+=1
                continue
            num_start = rest_of_str.find('+')
            non_anonymous_id = rest_of_str[num_start:num_start+16] # Making the data anonymous.
            if non_anonymous_id not in dict_of_id:
                dict_of_id[non_anonymous_id] = (len(dict_of_id)+1)
            my_dict_to_sort["id"] = dict_of_id[non_anonymous_id]
            my_dict_to_sort["text"] = rest_of_str
            list_of_chat_dicts.append(my_dict_to_sort)
            
    return list_of_chat_dicts

def lines_fixed(list_of_lines):
    fixed_lines_list =[]
    j=0  # j will be my fixed list counter
    for i,line in enumerate(list_of_lines): # i will be my original list counter
        is_new_message = line.find('.2021')
        if is_new_message<0:
            fixed_lines_list[j-1]+=line
            j-=1
        else:
            fixed_lines_list.append(line)
        j+=1
    return fixed_lines_list


def meta_data(list_of_dicts):
    data_dict = list_of_dicts[0]
    my_inner_dict = dict()
    my_text = data_dict['text']
    group_name_start = my_text.find('"')
    rest_of_str = my_text[group_name_start+1:]
    group_name_end = rest_of_str.find('"')
    group_name = rest_of_str[:group_name_end]
    my_inner_dict["chat_name"] = group_name
    my_inner_dict["creation_date"] = data_dict['datetime']
    num_of_participants = 1
    for a_dict in list_of_dicts:
        if a_dict['id']>num_of_participants:
            num_of_participants = a_dict['id']
    my_inner_dict["num_of_participants"] = num_of_participants
    my_inner_dict["creator"] = data_dict['id']
    return my_inner_dict


def all_data_dict(list_of_chat_dicts,dict_of_meta_data):
    total_dict = dict()
    total_dict["meta_data"] = dict_of_meta_data
    total_dict["messages"] = list_of_chat_dicts
    return total_dict


import json

my_path = input('Please write your file_path like this -> file_path.txt   ') # 'whatsapp_chat.txt'
my_list_of_chat_and_data_dicts = my_chat_reader(my_path)
my_list_of_chat_dicts = my_list_of_chat_and_data_dicts[1:]
meta_data_dict = meta_data(my_list_of_chat_and_data_dicts)
my_all_data_dict = all_data_dict(my_list_of_chat_dicts,meta_data_dict)

my_file_name = meta_data_dict["chat_name"]
my_file_name+='.json'
open_a_new_file = open(my_file_name, 'a', encoding='utf-8')
json.dump(my_all_data_dict, open_a_new_file, indent = 4, ensure_ascii=False)
open_a_new_file.close()

Please write your file_path like this -> file_path.txt   whatsapp_chat.txt
{
    "meta_data": {
        "chat_name": "יום הולדת בנות לנוי'ה",
        "creation_date": "6.4.2021, 11:39",
        "num_of_participants": 13,
        "creator": 1
    },
    "messages": [
        {
            "datetime": "6.4.2021, 11:58",
            "id": 2,
            "text": " ליה תגיע😍"
        },
        {
            "datetime": "6.4.2021, 12:09",
            "id": 3,
            "text": " מזל טוב, אמה תגיע"
        },
        {
            "datetime": "6.4.2021, 12:09",
            "id": 1,
            "text": " תודה❤️❤️"
        },
        {
            "datetime": "6.4.2021, 12:23",
            "id": 4,
            "text": " אשמח לפרטים.  לא ראיתי את ההזמנה😊"
        },
        {
            "datetime": "6.4.2021, 12:30",
            "id": 5,
            "text": " המון מזל טוב ♥️♥️♥️לין תגיע בע\"ה"
        },
        {
            "datetime": "6.4.2021, 12:30",
            "id": 1,
            "t